<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/Sentiment_Analysis_on_Movie_Reviews_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis on Movie Reviews Bayes:
[https://github.com/GaoangLiu/AA_ipynb/blob/master/Sentiment_Analysis_on_Movie_Reviews_Naive_Bayes.ipynb](https://github.com/GaoangLiu/AA_ipynb/blob/master/Sentiment_Analysis_on_Movie_Reviews_Naive_Bayes.ipynb)

## import packages

In [0]:
import math
import re
import os
import timeit
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import logging
import time
import smart_open
import importlib

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
logging.basicConfig(format='[%(asctime)s %(levelname)8s] %(message)s', level=logging.INFO, datefmt='%m-%d %H:%M:%S')

from keras import layers, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, Model, load_model
from keras.layers import Flatten, Dense, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api

from tqdm.notebook import tqdm
import tensorflow_hub as tfh

## Download files 

In [0]:
! rm *.tsv *.zip *.csv
! wget -O movie.zip ali.140714.xyz:8000/sentiment_analysis.zip 
! wget -O b7.py ali.140714.xyz:8000/boost117.py
! unzip movie.zip 
! ls

## Tune models


In [0]:
class Classifier():
  def __init__(self):
    self.train = None
    self.test = None 
    self.model = None
    
  def load_data(self, train_file='train.csv', test_file='test.csv'):
      """ Load train, test csv files and return pandas.DataFrame
      """
      self.train = pd.read_csv('train.tsv', sep="\t")
      self.train.rename({'Phrase': 'text', 'Sentiment': 'target'}, axis='columns', inplace=True)
      self.test = pd.read_csv('test.tsv', sep="\t")
      self.test.rename({'Phrase': 'text', 'Sentiment': 'target'}, axis='columns', inplace=True)
      logging.info('TSV data loaded')
  
  def countvectorize(self):
    #   tv = TfidfVectorizer(ngram_range=(1,5), token_pattern=r'\w{1,}',
    #            min_df=2, max_df=0.9, strip_accents='unicode', 
    #            smooth_idf=1, sublinear_tf=1, max_features=5000)
      tv = CountVectorizer()
      tv.fit(pd.concat([self.train.text, self.test.text]))
      self.vector_train = tv.transform(self.train.text)
      self.vector_test  = tv.transform(self.test.text)
      logging.info("Train & test text tokenized")

  def train_model(self):
      # Choose your own classifier: self.model and run it
      logging.info(f"{self.__class__.__name__} starts running.")
      labels = self.train.target
      X_train, X_val, y_train, y_val = train_test_split(self.vector_train, labels, test_size=0.2, random_state=2020)
      self.model.fit(X_train, y_train)
      
      val_preds = self.model.predict(X_val)
      print(f"Accuracy score: ", accuracy_score(y_val, val_preds))

      self.X_val, self.y_val = X_val, y_val
      return self.model

  def save_predictions(self, y_preds):
      sub = pd.read_csv(f"sampleSubmission.csv")
      sub['Sentiment'] = y_preds 
      sub.to_csv(f"submission_{self.__class__.__name__}.csv", index=False)
      logging.info(f'Prediction exported to submission_{self.__class__.__name__}.csv')
  
  def pipeline(self):
      s_time = time.clock()
      self.load_data()
      self.countvectorize()
      self.build_model()
      self.train_model()
      logging.info(f"Program running for {time.clock() - s_time} seconds")
